# FIBO MCP Ablation Study

Compare Claude Opus 4.5 and GPT-5.2 responses with and without FIBO MCP.

In [1]:
QUESTIONS = [
    "What is money?",
    "What is a country?",
    "What is corporate?",
]

In [ ]:
import os
import json
import asyncio
from pathlib import Path

import pandas as pd

from openai import AsyncOpenAI
from claude_agent_sdk import ClaudeAgentOptions, query
from claude_agent_sdk.types import AssistantMessage, ResultMessage, TextBlock, ToolUseBlock

from dotenv import load_dotenv
load_dotenv()

# Remote MCP endpoint
FIBO_MCP_URL = os.environ["FIBO_MCP_URL"]
FIBO_MCP_KEY = os.environ["FIBO_MCP_KEY"]

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
# use `claude setup-token` to get a new OAuth token
CLAUDE_CODE_OAUTH_TOKEN = os.environ["CLAUDE_CODE_OAUTH_TOKEN"]

In [ ]:
# claude-agent-sdk is equivalent to claude code

async def query_claude_code(question: str, use_fibo: bool=False) -> str:
    """Query Claude with FIBO MCP tool access."""
    mcp_servers = None
    if use_fibo:
        mcp_servers = {
            "fibo-mcp": {
                "type": "http",
                "url": FIBO_MCP_URL,
                "headers": {"Authorization": f"Bearer {FIBO_MCP_KEY}"},
            },
        }
    options = ClaudeAgentOptions(
        model="claude-opus-4-5-20251101",
        system_prompt="Use fibo MCP tool to answer questions about financial concepts.",
        mcp_servers=mcp_servers,
        allowed_tools=["mcp__fibo-mcp__*"],
        permission_mode="bypassPermissions",
    )
    async for message in query(prompt=question, options=options):
        if isinstance(message, ResultMessage):
            return message.result or ""
        
    return ""

In [ ]:
openai_client = AsyncOpenAI(api_key=OPENAI_API_KEY)

async def query_gpt52(question: str, use_fibo: bool=False) -> str:
    """Query GPT-5.2 with FIBO MCP tool access."""
    tools = None
    if use_fibo:
        tools = [
            {
                "type": "mcp",
                "server_label": "fibo",
                "server_url": FIBO_MCP_URL,
                "headers": {"Authorization": f"Bearer {FIBO_MCP_KEY}"},
                "require_approval": "never",
            },
        ]
    response = await openai_client.responses.create(
        model="gpt-5.2",
        instructions="Use fibo MCP tool to answer questions about financial concepts.",
        tools=tools,
        input=question,
    )
    return response.output_text

In [17]:
def write_jsonl(results:list, llm_provider:str):
    with open(f"{llm_provider}_results.jsonl", "w") as f:
        for i, question in enumerate(QUESTIONS):
            # Without FIBO
            f.write(json.dumps({
                "question": question,
                "with_fibo": False,
                "answer": results[i*2]
            }) + "\n")

            # With FIBO
            f.write(json.dumps({
                "question": question,
                "with_fibo": True,
                "answer": results[i*2+1]
            }) + "\n")

In [34]:
from IPython.display import display, Markdown

def display_markdown(results: list):
    md = []

    for i, question in enumerate(QUESTIONS):
        # Question header
        md.append(f"# {question}\n")

        # Side-by-side comparison using HTML table
        md.append('<table>')
        md.append('<tr>')
        md.append('<th width="50%">Raw</th>')
        md.append('<th width="50%">With FIBO MCP</th>')
        md.append('</tr>')
        md.append('<tr>')
        md.append(f'<td valign="top">\n\n{results[i*2]}\n\n</td>')
        md.append(f'<td valign="top">\n\n{results[i*2+1]}\n\n</td>')
        md.append('</tr>')
        md.append('</table>\n')

    display(Markdown('\n'.join(md)))

In [5]:
tasks = []
for question in QUESTIONS:
    tasks.append(query_gpt52(question))
    tasks.append(query_gpt52(question, use_fibo=True))

openai_results = await asyncio.gather(*tasks)

In [35]:
display_markdown(openai_results)

# What is money?

<table>
<tr>
<th width="50%">Raw</th>
<th width="50%">With FIBO MCP</th>
</tr>
<tr>
<td valign="top">

Money is anything that a society widely accepts as payment for goods and services and for settling debts.

Economists usually describe money by the key functions it serves:

1. **Medium of exchange**: It’s used to buy and sell things, avoiding barter (e.g., you don’t need to trade bread for shoes directly).
2. **Unit of account**: It provides a common measure for prices and value (e.g., dollars, euros), making it easier to compare goods and keep financial records.
3. **Store of value**: It can be saved and used later (though inflation or risk can reduce its purchasing power over time).
4. **Standard of deferred payment** (often included): It’s used to denominate debts and contracts to be paid in the future (rent, loans, wages).

Money can take many forms—**cash**, **bank deposits** (most modern money), and sometimes **digital forms** like mobile money or central bank digital currencies. What makes it “money” isn’t the physical form; it’s the **shared trust and acceptance**, backed by institutions, law, and economic stability.

</td>
<td valign="top">

Money is **anything that a community widely accepts as payment and as a way to measure and store value**.

In economics, money is usually defined by three core functions:

1. **Medium of exchange** – used to buy and sell goods and services (so people don’t have to barter).
2. **Unit of account** – provides a common yardstick for prices and debts (e.g., “$10”).
3. **Store of value** – can be saved and used later (though its purchasing power can rise/fall with inflation).

### Common forms of money
- **Currency (cash):** notes and coins.
- **Bank money:** balances in checking/savings accounts; most modern “money” is this.
- **Central bank reserves:** money banks use to settle payments between themselves (not typically used directly by the public).

### What makes something “money”?
It’s largely a matter of **collective trust and acceptance**, often supported by law and institutions (e.g., governments and central banks). For example, a dollar bill has value not because of the paper, but because it’s widely accepted and backed by the monetary system.

If you tell me whether you mean “money” in everyday life, in economics, or in finance/ontology terms, I can tailor the definition.

</td>
</tr>
</table>

# What is a country?

<table>
<tr>
<th width="50%">Raw</th>
<th width="50%">With FIBO MCP</th>
</tr>
<tr>
<td valign="top">

A **country** is a political territory recognized as a distinct place where a **government** has authority over a **population** within defined **borders**.

In practice, a place is commonly considered a country if it has most of these features:

- **Defined territory** (land and/or water with borders, even if disputed)
- **Permanent population**
- **Government** that controls internal affairs
- **Sovereignty** (ability to make its own laws and decisions without another state’s authority)
- **Capacity to conduct foreign relations** (e.g., make treaties, join international organizations)
- Often (but not always) **international recognition** by other countries

Important nuance: some places function like countries but aren’t widely recognized (e.g., Taiwan, Kosovo by some states), and some “countries” are parts of larger states (e.g., Scotland is called a country within the UK). Definitions can depend on legal, political, and diplomatic context.

</td>
<td valign="top">

In FIBO (the Financial Industry Business Ontology), what people commonly call a **country** is modeled as a **Sovereign State**.

**Definition (FIBO: `fibo:BE/GovernmentEntities/GovernmentEntities/SovereignState`):**  
A **sovereign state** is a **non-physical juridical entity** represented by **one centralized government** that has **sovereignty over a geographic area**.

So, a “country” is essentially a legally recognized political entity (not the land itself) whose government exercises supreme authority over a defined territory.

</td>
</tr>
</table>

# What is corporate?

<table>
<tr>
<th width="50%">Raw</th>
<th width="50%">With FIBO MCP</th>
</tr>
<tr>
<td valign="top">

“Corporate” refers to anything related to a **corporation**—a legally formed company recognized as a separate entity from its owners.

Common uses:
- **Corporate (adjective):**  
  - *Corporate office* = a company’s main headquarters  
  - *Corporate policy* = rules set by company leadership  
  - *Corporate culture* = shared norms/values in a company  
  - *Corporate finance* = money management for companies
- **Corporate world / corporate sector:** Large businesses and the environment of professional office-based work.
- **Corporate (noun, less common):** A group of corporations or people acting as a body (rare in everyday speech).

In casual speech, “corporate” can also imply something **formal, bureaucratic, or profit-focused**, especially associated with large companies.

</td>
<td valign="top">

“Corporate” is an adjective that means **relating to a corporation**—i.e., a **legally incorporated organization** recognized as a separate legal person from its owners/shareholders.

In practice, “corporate” is used to describe things connected to that kind of business entity, for example:

- **Corporate governance**: how a corporation is directed and controlled (board, executives, policies).
- **Corporate finance**: how a corporation raises and uses money (equity, debt, dividends, investment).
- **Corporate actions**: events a corporation initiates that affect its securities (e.g., dividends, stock splits, mergers).
- **Corporate bonds**: bonds issued by corporations (as opposed to government bonds).

Meaning depends on context: sometimes people use “corporate” more loosely to mean “big-business-like,” but the precise meaning is “of or pertaining to a corporation.”

</td>
</tr>
</table>


In [16]:
tasks = []
for question in QUESTIONS:
    tasks.append(query_claude_code(question))
    tasks.append(query_claude_code(question, use_fibo=True))

claude_code_results = await asyncio.gather(*tasks)

In [36]:
display_markdown(claude_code_results)

# What is money?

<table>
<tr>
<th width="50%">Raw</th>
<th width="50%">With FIBO MCP</th>
</tr>
<tr>
<td valign="top">

Money is a medium of exchange that serves several key functions in an economy:

## Core Functions of Money

1. **Medium of Exchange** - Money facilitates transactions by eliminating the need for barter (directly trading goods for goods). Instead of needing a "double coincidence of wants," you can sell your goods for money and use that money to buy what you need.

2. **Unit of Account** - Money provides a common measure of value, allowing us to compare the worth of different goods and services (e.g., "this costs $10, that costs $50").

3. **Store of Value** - Money allows you to save purchasing power for the future. You can earn money today and spend it later.

4. **Standard of Deferred Payment** - Money enables credit and debt, allowing payments to be made in the future.

## Types of Money

- **Commodity Money** - Has intrinsic value (e.g., gold, silver, salt historically)
- **Fiat Money** - Has no intrinsic value but is declared legal tender by a government (e.g., US dollars, euros)
- **Digital/Electronic Money** - Exists in bank accounts, digital payment systems
- **Cryptocurrency** - Decentralized digital currencies like Bitcoin

## Key Properties of Good Money

- **Durability** - Doesn't deteriorate quickly
- **Portability** - Easy to carry and transfer
- **Divisibility** - Can be broken into smaller units
- **Uniformity** - Each unit is the same as another
- **Scarcity** - Limited supply to maintain value
- **Acceptability** - Widely recognized and accepted

Money is essentially a social construct—it works because people collectively agree to accept it as payment and trust in its value.

</td>
<td valign="top">

Now I have comprehensive information from FIBO. Let me provide you with a complete answer.

---

## What is Money?

According to **FIBO** (Financial Industry Business Ontology) — the standard ontology used by major banks and financial regulators — money is represented through several interconnected concepts:

### 1. **Currency** (The Core Concept)
> **Definition**: *"Medium of exchange value, defined by reference to the geographical location of the monetary authorities responsible for it"*
> 
> — Source: ISO 4217 (Codes for the representation of currencies and funds)

Currency is also known as:
- **Currency unit**
- **Monetary unit**

In FIBO, Currency is classified as a **Measurement Unit** — it's the unit used to measure and express value.

---

### 2. **Amount of Money**
> **Definition**: *"Amount of readily available cash in banknotes and coins"*

This represents the physical manifestation of money — the actual cash you can hold and use.

---

### 3. **Monetary Amount**
> **Definition**: *"A measure that is an amount of money specified in monetary units"*

This is a more abstract concept — a quantified value expressed in a particular currency (e.g., $100, €50, ¥10,000).

---

### 4. **Cash Flow**
> **Definition**: *"The movement of money from some source to some sink"*

This captures how money functions dynamically in the economy — flowing between parties in transactions.

---

### Key Characteristics of Money in FIBO

| Characteristic | FIBO Representation |
|---------------|---------------------|
| **Medium of Exchange** | Currency serves as the medium for exchanging value |
| **Unit of Account** | Monetary units provide a standard measure of value |
| **Issued by Authority** | Defined by geographical monetary authorities (central banks) |
| **Physical Form** | Banknotes and coins (Amount of Money) |
| **Abstract Form** | Monetary amounts specified in currency units |

---

### Related Financial Concepts

FIBO also defines many money-related instruments and services:
- **Money Market Instruments** — Short-term debt securities
- **Money Services Business** — Entities that deal in currency exchange, money transmission, etc.
- **Currency Instruments** — Financial instruments used for currency trading
- **Cash Settlement** — Settlement of financial contracts in money rather than physical assets

In summary, FIBO models **money** as a **medium of exchange** represented by **currency units** issued by **monetary authorities**, which can exist as physical **cash** or abstract **monetary amounts**.

</td>
</tr>
</table>

# What is a country?

<table>
<tr>
<th width="50%">Raw</th>
<th width="50%">With FIBO MCP</th>
</tr>
<tr>
<td valign="top">

A **country** is a distinct territorial body or political entity that is recognized as an independent nation. Countries are characterized by several key features:

## Key Characteristics

1. **Defined Territory** - A country has recognized geographic boundaries that separate it from other countries.

2. **Permanent Population** - People who live within its borders on an ongoing basis.

3. **Government** - A political system that manages the country's affairs, makes laws, and provides services to its citizens.

4. **Sovereignty** - The authority to govern itself without interference from other countries. This is often considered the most important characteristic.

5. **Recognition** - Other countries acknowledge its existence as an independent nation (though this can be disputed in some cases).

## Related Terms

- **Nation** - Often used interchangeably with "country," but technically refers to a group of people who share common identity, culture, language, or ethnicity.
- **State** - In international law, this term is synonymous with "country" (not to be confused with subdivisions like U.S. states).
- **Nation-state** - A country where the nation (cultural group) and state (political entity) align closely.

## Examples

There are approximately **195 countries** in the world today, including:
- Large countries like Russia, Canada, and the United States
- Small countries like Vatican City, Monaco, and San Marino
- Island nations like Japan, Australia, and Madagascar

Countries can vary enormously in size, population, government type, wealth, and culture.

</td>
<td valign="top">

Based on the **Financial Industry Business Ontology (FIBO)**, here's what a **country** is:

## Country (Sovereign State) in FIBO

In FIBO, a "country" is formally modeled as a **Sovereign State**, defined as:

> **"A non-physical juridical entity that is represented by one centralized government that has sovereignty over a geographic area."**

### Key Characteristics

A sovereign state (country) has four essential elements according to FIBO:

1. **One centralized government** - A single governing authority
2. **Permanent population** - People who inhabit the territory
3. **Defined territory** - Clear geographic boundaries
4. **Capacity to enter into relations with other sovereign states** - The ability to engage in international diplomacy and agreements

### Ontological Classification

In the FIBO hierarchy:
- A **Sovereign State** is a type of **Polity**
- A **Polity** is defined as a *"legal person that is a supranational entity, crown, state, or subordinate civil authority (such as a province, prefecture, county, municipality, city, or district) representing the people of that entity"*

### Financial Relevance

Countries are fundamental concepts in finance because they:
- Issue **sovereign bonds** and **sovereign debt instruments**
- Operate **central banks** and **monetary authorities**
- Define **jurisdictions** for financial regulation
- Issue **currencies**
- Host **national governments** that regulate financial institutions

</td>
</tr>
</table>

# What is corporate?

<table>
<tr>
<th width="50%">Raw</th>
<th width="50%">With FIBO MCP</th>
</tr>
<tr>
<td valign="top">

The term "corporate" can have several meanings depending on the context:

## As an Adjective

1. **Relating to a corporation or business**: Describes things associated with large companies or businesses
   - "Corporate headquarters" - the main office of a company
   - "Corporate culture" - the values, beliefs, and practices within a company
   - "Corporate policy" - rules and guidelines set by a company

2. **Collective or unified**: Relating to a group acting as one body
   - "Corporate responsibility" - shared responsibility of a group

## Common Uses

- **Corporate world/sector**: The business environment, especially large companies
- **Corporate law**: Legal matters relating to businesses and corporations
- **Corporate governance**: How companies are directed and controlled
- **Corporate identity**: A company's brand, image, and visual representation
- **Corporate ladder**: The hierarchy of positions within a company (as in "climbing the corporate ladder")

## In Casual/Slang Usage

"Corporate" is sometimes used pejoratively to describe something as:
- Overly commercial or profit-driven
- Lacking authenticity or soul
- Bureaucratic or impersonal

---

**Were you asking about "corporate" in a specific context?** For example:
- A concept in business/law?
- A file or function in a codebase you're working with?
- Something else?

Let me know and I can provide more targeted information!

</td>
<td valign="top">

Based on my research in FIBO (Financial Industry Business Ontology), here's what **"corporate"** means in the financial industry:

## Corporation (Core Definition)

A **corporation** is defined in FIBO as:

> *"A formal organization that is a legal entity (artificial person) distinct from its owners, created under the jurisdiction of the laws of a state or nation."*

### Three Distinguishing Characteristics:

1. **Separation of ownership from management and liability** - Its liability to creditors is limited to its resources (unlike partnerships and sole proprietorships)
2. **Ability to negotiate contracts and own property** - The corporation can act as a legal person
3. **Transferable ownership** - Ownership can change irrespective of changes in membership or the lifetimes of its stockholders

A corporation is also known as a **"body corporate"** and is managed by a **board of directors** that determines corporate policy, while officers manage day-to-day affairs.

---

## Key Corporate-Related Concepts in FIBO:

| Concept | Definition |
|---------|------------|
| **Stock Corporation** | A corporation that has shareholders who receive ownership through shares of stock, with potential dividend returns and voting rights |
| **Corporate Bond** | A bond issued by a company to raise financing for purposes like M&A, expansion, or operational needs (maturity ≥1 year) |
| **Corporate Action** | Events initiated by a corporation that affect its securities or shareholders |
| **Corporate Officer** | An executive responsible for managing the corporation |
| **Corporate Bylaws** | Rules governing the corporation's internal operations |
| **Articles of Incorporation** | Legal documents filed to establish a corporation |
| **Benefit Corporation** | A type of corporation with expanded purpose beyond profits |

The term "corporate" in finance generally relates to activities, instruments, and governance structures associated with corporations as legal business entities.

</td>
</tr>
</table>
